# Tunneleffekt

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden wir uns drei verschiedene Szenarien ansehen, in denen der Tunneleffekt eine Rolle spielt.

1. Tunneleffekt an einer Rechteck-Barriere   
   Hier sehen wir uns die Dynamik eines Wellenpakets an, das auf eine Rechteck-Barriere trifft. Abgesehen von der Anwesenheit der 
   Barriere entspricht das Vorgehen demjenigen bei der Betrachtung des freien Wellenpakets in
   [5-05-Freies-Teilchen.ipynb](5-05-Freies-Teilchen.ipynb). Neben einer Darstellung der zeitabhängigen 
   Aufenthaltswahrscheinlichkeit als Animation sehen wir uns den zeitlichen Verlauf der Wahrscheinlichkeit an, das Teilchen in 
   einem der drei Teilbereiche konstanten Potentials zu finden.
   
2. Tunneln im Doppeltopfpotential   
   Auch hier betrachten wird die Dynamik eines anfänglichen gaußförmigen Wellenpakets, das sich nun aber in einem 
   Doppeltopfpotential bewegt. Dessen Parameter sind so gewählt, dass zwei gleich tiefe Potentialminima vorliegen. Das Tunneln
   zwischen den beiden Potentialminima durch die dazwischen liegende Potentialbarriere untersuchen wir anhand der 
   Aufenthaltswahrscheinlichkeit im linken und rechten Teil des Potentials.
   
3. Metastabilität durch Tunneln   
   Im letzten Teil wenden wir uns einem kubischen Potential zu, dessen Parameter so gewählt sind, dass ein Potentialminimum 
   vorliegt. Dieses ist durch eine Potentialbarriere von einem Bereich getrennt, in dem das Potential gegen $-\infty$ strebt. 
   Anfänglich liege im Potentialminimum ein gaußförmiges Wellenpaket vor, dessen Norm aufgrund des Tunneleffekts abnimmt. Aus der
   zeitlichen Änderung lässt sich eine Zerfallsrate bestimmen.

### Importanweisungen

*Hinweis:* Sollte die Animation nicht funktionieren, stellen Sie sicher, dass Sie dieses Jupyter-Notebook in JupyterLab ausführen.

In [ ]:
from math import pi, sqrt
import numpy as np
from scipy import fft, stats
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt
from matplotlib import animation

plt.style.use("numphyspy.style")

## Tunneleffekt an einer Rechteck-Barriere

In dimensionslosen Variablen ist die Rechteck-Barriere durch das Potential

$$V(x) = \begin{cases} 1 & \text{für $|x| < a$} \\ 0 & \text{sonst} \end{cases}$$

gegeben. Um Randeffekte zu vermeiden, fügen wir wie in [5-05-Freies-Teilchen.ipynb](5-05-Freies-Teilchen.ipynb) ein geeignetes optisches Potential hinzu. Da wir auch hier die Split-Operator-Methode verwenden, können wir den Code vom freien Teilchen weitestgehend übernehmen und müssen nur darauf achten, wo nötig das Barrierenpotential zu berücksichtigen. An den Stellen, an denen wir Code unverändert übernehmen, verzichten wir auf eine weitere Diskussion des Codes.

### Transformation vom Orts- in den Impulsraum und zurück

In [ ]:
def position_to_momentum(psi_position):
    psi_momentum = fft.fft(psi_position)
    return psi_momentum

In [ ]:
def momentum_to_position(psi_momentum):
    psi_position = fft.ifft(psi_momentum)
    return psi_position

### Zeitschritt

In [ ]:
def time_step(psi_position, dt, v, k):
    psi_position = psi_position * np.exp(-1j*v*dt/2)
    psi_momentum = position_to_momentum(psi_position)
    psi_momentum = psi_momentum * np.exp(-1j*k**2/2*dt)
    psi_position = momentum_to_position(psi_momentum)
    psi_position = psi_position * np.exp(-1j*v*dt/2)
    return psi_position

### Lösung der Schrödingergleichung

Im Vergleich zur gleichnamigen Funktion in [5-05-Freies-Teilchen.ipynb](5-05-Freies-Teilchen.ipynb) muss hier zu Beginn neben dem optischen Potential noch das Barrierenpotential berücksichtigt werden. Um diesen Code weiter unten wiederverwenden zu können, wird hier als erstes Argument `potential` eine Funktion übergeben, die das Barrierenpotential definiert. In diesem Abschnitt wird beim Aufruf die Funktion `rectangular_barrier` im Argument `potential` übergeben.

In [ ]:
def time_development(potential, potential_parameter,
                     psi_position, n_time, dt, x_values, dx,
                     v_opt, sigma_opt, x_max):
    v = potential(x_values, potential_parameter,
                  v_opt, sigma_opt, x_max)
    xdim = x_values.shape[0]
    psi_squared_of_time = np.zeros((n_time+1, xdim))
    k = fft.fftfreq(xdim, dx) * 2*pi
    psi_squared_of_time[0, :] = np.absolute(psi_position)**2
    for n in range(n_time):
        psi_position = time_step(psi_position, dt, v, k)
        psi_squared_of_time[n+1, :] = (
            np.absolute(psi_position)**2)
    return psi_squared_of_time

### Erzeugung des Anfangszustands

Der Anfangszustand ist durch eine Gauß-Funktion gegeben, deren Zentrum, Breite und mittlerer Impuls durch die Parameter `x_0`, `sigma_0` und `k_0` festgelegt werden können. Der Zustandsvektor wird numerisch normiert.  

In [ ]:
def gaussian_wavepacket(x, x_0, sigma_0, k_0, dx):
    psi_position = np.exp(-0.25*((x-x_0)/sigma_0)**2
                          + 1j*k_0*x)
    norm = np.sum(np.absolute(psi_position)**2) * dx
    return psi_position / sqrt(norm)

### Definition des Potentials

Die Funktion `rectangular barrier` berücksichtigt neben der Rechteck-Barriere auch den Beitrag eines optischen Potentials.

In [ ]:
def v_optical(x, v, x_max, sigma):
    return -1j*v*(np.exp(-(-x+x_max)**2/(2*sigma**2))
                  + np.exp(-(-x-x_max)**2/(2*sigma**2)))

In [ ]:
def rectangular_barrier(x, a, v_opt, sigma_opt, x_max):
    v_barrier = np.where(np.abs(x) < a, 1, 0)
    v = v_barrier + v_optical(x, v_opt, x_max, sigma_opt)
    return v

### Implementierung des Bedienelements und animierte Darstellung der Aufenthaltswahrscheinlichkeit

Mit Hilfe des Schiebereglers lässt sich der folgende Parameter einstellen:
- `k_0`: mittlerer Impuls. Hiermit kann die mittlere kinetische Energie des einfallenden Wellenpakets variiert werden. Der voreingestellte Wert ist so gewählt, dass die zugehörige Energie knapp unterhalb der Barrierenhöhe liegt, die dem Wert $k_0=\sqrt{2}$ entspricht.

Weitere Parameter sind zu Beginn der Funktion `make_animation_rectangular_barrier` sinnvoll vorgegeben, können dort aber bei Bedarf verändert werden.

Für die graphische Darstellung wird zunächst die maximale Aufenthaltswahrscheinlichkeit `y_max` im gesamten Zeitintervall bestimmt. Durch die Festlegung der Achsenbegrenzungen mit `ax.set_xlim` und `ax.set_ylim` wird sichergestellt, dass die Achsen während der gesamten Animation unverändert bleiben. Mit dem Aufruf der Funktion `ax.fill` wird der Barrierenbereich grau markiert.

In [ ]:
k_0_widget = widgets.FloatSlider(
    value=1.4, min=0.1, max=1.5, step=0.01,
    description="$k_0$")

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(k_0=k_0_widget)
def make_animation_rectangular_barrier(k_0):
    barrier_width = 2
    v_opt = 2
    sigma_opt = 5
    t_end = 50/k_0
    n_time = 300
    n_max = 4096
    x_max = 50
    sigma_0 = 5
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               endpoint=False, retstep=True)
    psi_position = gaussian_wavepacket(x_values, -x_max/2,
                                       sigma_0, k_0, dx)
    psi_squared_of_time = time_development(
        rectangular_barrier, barrier_width/2, psi_position,
        n_time, t_end/n_time, x_values, dx, v_opt,
        sigma_opt, x_max)

    def init():
        line.set_data(x_values, psi_squared_of_time[0, :])
        return line,

    def animate(i):
        line.set_data(x_values, psi_squared_of_time[i, :])
        return line,

    clear_output()
    fig, ax = plt.subplots()
    line, = ax.plot([], [])
    ax.set_xlim((-x_max, x_max))
    y_max = np.max(psi_squared_of_time)
    ax.set_ylim((0, y_max))
    ax.fill([-1, -1, 1, 1], [0, y_max, y_max, 0], "#ccc")
    ax.set_xlabel(r"$\bar x$")
    ax.set_ylabel(r"$\vert\Psi(x, t)\vert^2$")
    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=n_time,
        interval=10, blit=True, repeat=False)
    plt.close()
    display(HTML(anim.to_jshtml()))

### Berechnung der Aufenthaltswahrscheinlichkeiten in den Bereichen I, II und III

Um die Aufenthaltswahrscheinlichkeiten in den Bereichen konstanten Potentials zu berechnen, werden die drei Regionen durch Vergleiche der verwendeten Gitterpositionen mit den Koordinaten der Barrierenränder bestimmt. Damit lassen sich die zeitabhängigen
Aufenthaltswahrscheinlichkeiten in diesen Bereichen leicht durch Summation über die Achse 1 bestimmen.

In [ ]:
def norms_rectangular_barrier(psi_squared_of_time, n_time,
                              x_values, dx, a):
    norms = []
    for region in (x_values <= -a,
                   np.abs(x_values) < a,
                   a <= x_values):
        norms.append(np.sum(psi_squared_of_time[:, region],
                            axis=1
                            )*dx)
    norms.append(sum(norms))
    return norms

### Implementierung des Bedienelements und graphische Darstellung der Aufenthaltswahrscheinlichkeiten in den Bereichen I, II und III

Mit Hilfe des Schiebereglers lässt sich der folgende Parameter einstellen:
- `k_0`: mittlerer Impuls. Hiermit kann die mittlere kinetische Energie des einfallenden Wellenpakets variiert werden. Der voreingestellte Wert ist so gewählt, dass die zugehörige Energie knapp unterhalb der Barrierenhöhe liegt, die dem Wert $k_0=\sqrt{2}$ entspricht.

Weitere Parameter sind zu Beginn der Funktion `plot_norms_rectangular_barrier` sinnvoll vorgegeben, können dort aber bei Bedarf verändert werden.

Neben den Aufenthaltswahrscheinlichkeiten in den Bereichen I, II und III wird die Norm des Gesamtzustands graphisch dargestellt. Damit lässt sich beurteilen, wann Teile der Wellenfunktion durch das optische Potential an den Rändern absorbiert werden. Außerdem wird zur leichteren Interpretation der Daten die Zeit angegeben, die das Zentrum des Wellenpakets bis zum Erreichen der Potentialbarriere benötigt.

In [ ]:
k_0_widget = widgets.FloatSlider(value=1.4,
                                 min=0.1, max=1.5, step=0.01,
                                 description="$k_0$")

@interact(k_0=k_0_widget)
def plot_norms_rectangular_barrier(k_0):
    barrier_width = 2
    v_opt = 2
    sigma_opt = 5
    t_end = 100
    n_time = 2000
    n_max = 4096
    x_max = 80
    sigma_0 = 5
    t_values, dt = np.linspace(0, t_end, n_time+1,
                               retstep=True)
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               endpoint=False, retstep=True)
    psi_position = gaussian_wavepacket(x_values, -x_max/2,
                                       sigma_0, k_0, dx)
    psi_squared_of_time = time_development(
        rectangular_barrier, barrier_width/2, psi_position,
        n_time, dt, x_values, dx, v_opt, sigma_opt, x_max)
    norms = norms_rectangular_barrier(
        psi_squared_of_time, n_time, x_values, dx,
        barrier_width/2)
    labels = ("$p_I$", "$p_{II}$", "$p_{III}$", "Norm")

    fig, ax = plt.subplots()
    for norm, label in zip(norms, labels):
        ax.plot(t_values, norm, label=label)
    ax.set_xlabel("$t$")
    ax.set_ylabel("$p$")
    ax.legend(bbox_to_anchor=(1, 1.01), loc="upper left")
    fig.suptitle("Zeit bis zum Erreichen der Barriere: "
                 f"{(x_max - barrier_width) / (2*k_0):5.1f}",
                 fontsize=10)

## Tunneleffekt im Doppeltopfpotential

In diesem Abschnitt betrachten wir das Tunneln im Doppeltopfpotential

$$V(x) = -\frac{1}{4} x^2 + \alpha^2 x^4\,$$

das zwei gleich tiefe Potentialminima an den Stellen

$$x_0 = \pm\frac{1}{2\sqrt{2}\alpha}$$

besitzt. In der Nähe der Minima lässt sich das Potential harmonisch durch

$$V(x_0+\Delta x) \approx -\frac{1}{64\alpha^2} +\frac{1}{2}(x-x_0)^2$$

nähern. Im Folgenden betrachten wir die Zeitentwicklung des Zustands

$$\Psi(x, 0) = \frac{1}{\pi^{1/4}}\exp\left(-\frac{(x+\frac{1}{2\sqrt{2}\alpha})^2}{2}\right)\,$$

der dem Grundzustand in der harmonischen Näherung der linken Potentialmulde entspricht.

### Definition des Potentials

Neben dem Doppeltopfpotential mit Parameter `alpha` wird hier auch das optische Potential berücksichtigt. Für das Argument `x` sind sowohl skalare Werte als auch NumPy-Arrays zulässig. 

In [ ]:
def double_well(x, alpha, v_opt, sigma_opt, x_max):
    v_real = -x**2/4 + alpha**2*x**4
    v = v_real + v_optical(x, v_opt, x_max, sigma_opt)
    return v

### Berechnung der Aufenthaltswahrscheinlichkeiten links und rechts der Potentialbarriere

Das Vorgehen entspricht derjenigen bei der Rechteck-Barriere, wobei hier nur zwei Bereiche links und rechts der Potentialbarriere bei $x=0$ zu definieren sind.

In [ ]:
def norms_double_well(psi_squared_of_time, n_time, x_values,
                      dx):
    norms = []
    for region in (x_values < 0,
                   x_values >= 0):
        norms.append(np.sum(psi_squared_of_time[:, region],
                            axis=1
                            )*dx)
    norms.append(sum(norms))
    return norms

### Implementierung der Bedienelemente und graphische Darstellung der Aufenthaltswahrscheinlichkeiten links und rechts der Potentialbarriere

Mit Hilfe der Schiebregler lassen sich die folgenden Parameter einstellen:
- `alpha`: Potentialparameter
- `t_end`: Länge des zu betrachtenden Zeitintervalls

Weitere Parameter sind zu Beginn der Funktion `plot_norms_double_well` mit sinnvollen Werten vorbelegt, die dort bei Bedarf geändert werden können. Auch wenn Randeffekte wegen des quartischen Potentials keine große Rolle spielen, wie man am zeitlichen Verhalten der Gesamtnorm sehen kann, wird ein optisches Potential an den Rändern hinzugefügt.

In der graphischen Darstellung der zeitabhängigen Aufenthaltswahrscheinlichkeiten in den Bereichen links und rechts der Potentialbarriere ist eine Oszillation zu erkennen, die sich qualitativ mit Hilfe der beiden niedrigsten Eigenzustände verstehen lässt. Weitere Strukturen weisen auf den Einfluss höher angeregter Zustände hin.

In [ ]:
widget_dict = {"alpha":
               widgets.FloatSlider(
                   value=0.15, min=0.1, max=0.5, step=0.01,
                   description=r"$\alpha$"),
               "t_end":
               widgets.FloatLogSlider(
                   value=200, min=1, max=4, step=0.01,
                   description=r"$t_\text{end}$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_norms_double_well(alpha, t_end):
    v_opt = 2
    sigma_opt = 5
    n_time = int(t_end*10)
    n_max = 2048
    x_max = 80
    t_values, dt = np.linspace(0, t_end, n_time+1,
                               retstep=True)
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               endpoint=False, retstep=True)
    psi_position = gaussian_wavepacket(
        x_values, -1/(2*sqrt(2)*alpha), 1/sqrt(2), 0, dx)
    psi_squared_of_time = time_development(
        double_well, alpha, psi_position, n_time, dt,
        x_values, dx, v_opt, sigma_opt, x_max)
    norms = norms_double_well(psi_squared_of_time, n_time,
                              x_values, dx)
    labels = ("Norm links", "Norm rechts", "Norm")
    fig, ax = plt.subplots()
    for norm, label in zip(norms, labels):
        ax.plot(t_values, norm, label=label)
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\langle \Psi \vert \Psi \rangle$")
    ax.legend(bbox_to_anchor=(1, 1.01), loc="upper left")

## Metastabilität durch Tunneln

Abschließend betrachten wir das kubische Potential

$$V(x) = \frac{1}{2} x^2 - \alpha x^3\,,$$

das eine Mulde bei $x=0$ aufweist, die durch eine Potentialbarriere der Höhe $1/54\alpha^2$ bei $x=1/3\alpha$ von einem Bereich getrennt ist, in dem das Potential für $x\to\infty$ immer schneller abfällt. Auch hier interessieren wir uns für einen Anfangszustand, der dem Grunzustand im bei $x=0$ harmonisch genäherten Potential entspricht, und bestimmen aus dem zeitlichen Abfall der Aufenthaltswahrscheinlichkeit links der Potentialbarriere eine Zerfallsrate.

### Definition des Potentials

Dem kubischen Potential kann hier auch wieder ein optisches Potential an den Rändern hinzugefügt werden.

In [ ]:
def cubic_potential(x, alpha, v_opt, sigma_opt, x_max):
    v_real = x**2/2 - alpha*x**3
    v = v_real + v_optical(x, v_opt, x_max, sigma_opt)
    return v

### Berechnung der Norm als Funktion der Zeit sowie der Zerfallsrate

Nach der Berechnung der zeitabhängigen Aufenthaltswahrscheinlichkeit links der Barriere wird der Logarithmus dieser Werte in `log_norm_of_t` abgespeichert. Mit Hilfe der Funktion `stats.linregress` aus dem SciPy-Paket wird für die logarithmischen Daten eine Ausgleichsgerade berechnet, deren Steigung bis auf das Vorzeichen die Zerfallsrate `decay_rate` liefert.

In [ ]:
def norm_and_decay_rate(psi_squared_of_time, t_values,
                        x_values, dx, alpha):
    well_region = x_values < 1/(3*alpha)
    norm_of_t = np.sum(psi_squared_of_time[:, well_region],
                       axis=1) * dx
    log_norm_values = np.log(norm_of_t)
    lr_result = stats.linregress(t_values, log_norm_values)
    decay_rate = -lr_result.slope
    return norm_of_t, decay_rate

### Implementierung des Bedienelements und graphische Darstellung der Aufenthaltswahrscheinlichkeit links der Potentialbarriere

Mit Hilfe des Schiebereglers lässt sich der folgende Parameter einstellen:
- `alpha`: Potentialparameter

Auch hier werden wieder einige Parameter zu Beginn der Funktion `plot_norm_cubic_potential` auf sinnvolle Werte eingestellt.

Die Zeitabhängigkeit der Aufenthaltswahrscheinlichkeit links der Potentialbarriere wird in einer linear-logarithmischen Darstellung gezeigt, für die ein exponentieller Zerfall einem linearen Kurvenverlauf entspricht. Zudem wird die berechnete Zerfallsrate ausgegeben.

In [ ]:
alpha_widget = widgets.FloatSlider(
    value=0.2, min=0.01, max=0.5, step=0.01,
    description=r"$\alpha$")

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(alpha=alpha_widget)
def plot_norm_cubic_potential(alpha):
    v_opt = 2
    sigma_opt = 5
    t_end = 1000
    n_time = 10000
    n_max = 4096
    x_max = 20
    t_values, dt = np.linspace(0, t_end, n_time+1,
                               retstep=True)
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max,
                               endpoint=False, retstep=True)
    psi_position = gaussian_wavepacket(
        x_values, 0, 1/sqrt(2), 0, dx)
    psi_squared_of_time = time_development(
        cubic_potential, alpha, psi_position, n_time, dt,
        x_values, dx, v_opt, sigma_opt, x_max)
    norm_of_t, decay_rate = norm_and_decay_rate(
        psi_squared_of_time, t_values, x_values, dx, alpha)

    fig, ax = plt.subplots()
    ax.semilogy(t_values, norm_of_t,
                label=f"Zerfallsrate = {decay_rate:8.5f}")
    ax.set_xlabel("$t$")
    ax.set_ylabel("$p$")
    ax.legend(loc="lower left")